In [1]:
import cv2
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import math

from utils.line import *

In [2]:
img_path = "/home/rp24/file/test_img/3_1.png"
lower_hsv = np.array([0, 0, 240])   # 直线色域 0 0 245
upper_hsv = np.array([180, 255, 255])


frame = cv2.imread(img_path)
height, width, _ = frame.shape


lines = get_lines(frame, lower_hsv, upper_hsv)
ca = get_center_and_angle(lines)

ca.shape


(114, 3)

In [ ]:
plt.imshow(frame)

# Add lines on the frame
num_start = 0
num_end = num_start + 44
for idx in range(lines[num_start:num_end, ...].shape[0]):
    x0, y0, x1, y1 = lines[idx][0]
    center, angle = calculate_line_center_and_angle(x0, y0, x1, y1)
    print(f"center: {center}, angle: {angle:.1f}, {x0}, {y0} {x1}, {y1}")
    plt.plot([x0, x1], [y0, y1], color='red', linewidth=2)

# Show the plot
plt.show()

# 聚类

In [ ]:
keylines = lines[:10, ...]
keylines = keylines.reshape(-1, 4)
keylines.shape

In [ ]:
# from sklearn.cluster import KMeans

# # Create a KMeans object
# kmeans = KMeans(n_clusters = 3)

# # Fit the KMeans model to the line coordinates
# kmeans.fit(keylines)

# # Get the cluster labels for each line
# line_labels = kmeans.labels_

line_labels = cluster(keylines)

# Print the cluster labels
# print(line_labels)
for idx in range(keylines.shape[0]):
    ca = calculate_line_center_and_angle(keylines[idx][0], 
                                         keylines[idx][1],
                                         keylines[idx][2],
                                         keylines[idx][3])
    print(f"{keylines[idx]} {ca}: {line_labels[idx]}")


# capture

In [ ]:
cap = cv.VideoCapture(0)

output_dir = "/home/rp24/file/debug_output"

idx = 0

In [ ]:
now

# ref

In [ ]:
img_path = "/home/rp24/file/test_img/3_1.png"
frame = cv.imread(img_path)
hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)

In [ ]:
def line_detection(image):
    img = image[range(0, 240, 4), :]  # 截取前半部分
    
    h, w, ch = img.shape
    redLower = np.array([0, 0, 230])
    redUpper = np.array([255, 255, 255])
    hsv = cv.cvtColor(img,cv.COLOR_BGR2HSV)  # 转换为HSV空间
    # 根据阈值构建掩膜
    mask = cv.inRange(hsv, redLower, redUpper)
    # 腐蚀操作
    mask = cv.erode(mask, None, iterations=1)
    # 将二值化图像转换为矩阵
    im = np.asarray(mask)

    # 按y轴扫描
    ins = np.zeros(h)
    im = np.insert(im, 0, ins, axis=1)
    x1 = (im != 0).argmax(axis=1)
    im_x2 = im[:, x1.max() + 30:x1.min():-1]
    x2 = (im_x2 != 0).argmax(axis=1)
    x2 = np.size(im_x2, 1) - x2 + x1.min()
    y = np.squeeze(np.nonzero(x1))
    x = np.array((x1[y] + x2[y]) / 2 - 1)
    try:
        if x[0] != x[-1]:
            k = (y[0] - y[-1]) / (x[0] - x[-1]) * 4
            a = round((cmath.atan(k) / 3.14 * 180).real) - 90
            if a <= -90:
                a += 180
        else:
            a = 0
        a += 90    
        d = int(160 - x[-1])
    except:
        a = 999
        d = 999
    return img,a,d